In [ ]:
from PRM2ORM import PRM2ORM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


# initialize variable
INPUT_PATH = 'src/en-zh'
# The name/path of the trained model and reference model.
TRAINED_PATH = 'sabijun/MT-PRM-LLaMA-3.2-3B'
REF_PATH = 'meta-llama/Llama-3.2-3B-Instruct'
# gpu index
trained_device = 'cuda:0'
ref_device = 'cuda:1'
# The method of Sequence-level score calculation.
# Possible choices: 'weighted', 'length normalized'
method = 'weighted'
# correspond with the translation direction
# e.g. from Chinese to English
language_pair = ['English', 'Chinese']

In [ ]:
# load the models
trained_model = AutoModelForCausalLM.from_pretrained(TRAINED_PATH, torch_dtype=torch.bfloat16).to(trained_device)
ref_model = AutoModelForCausalLM.from_pretrained(REF_PATH, torch_dtype=torch.bfloat16).to(ref_device)
tokenizer = AutoTokenizer.from_pretrained(REF_PATH)

In [ ]:
prm2orm = PRM2ORM(trained_model, trained_device, ref_model, ref_device, tokenizer)

In [ ]:
# read the sentence
f = open(INPUT_PATH, 'r', encoding='utf-8')
lines = f.read().splitlines()
f.close()

f = open('output/results', 'w', encoding='utf-8')
for i in range(len(lines) // 2):
    score = prm2orm.score(['English', 'Chinese'], lines[2*i], lines[2*i+1], method)
    f.write(lines[2*i] + '\n' + lines[2*i+1] + '\t' + str(score.item()) + '\n')
    print(lines[2*i] + '\n' + lines[2*i+1] + '\nscore:' + str(score.item()))
f.close()